## Setup

In [1]:
reset -fs

In [2]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

/home/vietpride12/anaconda3/envs/ml/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
RANDOM_STATE = 28

## Load Data and Filter

In [4]:
# Comments File
comments = '../Data/CommentsApril2018.csv'

In [ ]:
# Read in File
comm = pd.read_csv(comments)

/home/vietpride12/anaconda3/envs/ml/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
comm.columns

Index(['approveDate', 'articleID', 'articleWordCount', 'commentBody',
       'commentID', 'commentSequence', 'commentTitle', 'commentType',
       'createDate', 'depth', 'editorsSelection', 'inReplyTo', 'newDesk',
       'parentID', 'parentUserDisplayName', 'permID', 'picURL', 'printPage',
       'recommendations', 'recommendedFlag', 'replyCount', 'reportAbuseFlag',
       'sectionName', 'sharing', 'status', 'timespeople', 'trusted',
       'typeOfMaterial', 'updateDate', 'userDisplayName', 'userID',
       'userLocation', 'userTitle', 'userURL'],
      dtype='object')

In [ ]:
# Filter out Unknown & comment commentTypes & Non-NAs
filt = (comm.sectionName != "Unknown") & (comm.commentType == "comment") & comm.sectionName.notna()
data = comm.commentBody[filt]
sections = comm.sectionName[filt]

## Train/Test Split and Training

In [ ]:
train_data, test_data, train_target, test_target = train_test_split(data, sections, random_state=RANDOM_STATE)

In [ ]:
vectorizer = CountVectorizer(decode_error='ignore',
                             stop_words='english')
dt_model = Pipeline([('vec', vectorizer),
                       ('clf', DecisionTreeClassifier(random_state=RANDOM_STATE))])

In [ ]:
dt_model.fit(train_data, train_target)

Pipeline(memory=None,
     steps=[('vec', CountVectorizer(analyzer='word', binary=False, decode_error='ignore',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        s...        min_weight_fraction_leaf=0.0, presort=False, random_state=28,
            splitter='best'))])

## Predictions and Metric Evaluation

In [ ]:
dt_model.score(test_data, test_target)

0.6114223390015369

## Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_params = dict(vec__ngram_range=[(1,1), (1,2), (1,3)],
                   clf__criterion=['gini', 'entropy'])

In [ ]:
gs = GridSearchCV(estimator=dt_model,
                 param_grid=grid_params,
                 scoring='accuracy',
                 cv=5,
                 n_jobs=-1)

In [ ]:
gs.fit(train_data, train_target)

/home/vietpride12/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/model_selection/_split.py:626: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


In [ ]:
gs.best_params_

In [ ]:
gs.best_estimator_

In [ ]:
gs.score(test_data, test_target)

## With TdidfVectorizor

In [ ]:
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem.porter import PorterStemmer

In [ ]:
def tokenize(text):
    """
    Tokenize text and return a non-unique list of tokenized words
    found in the text. Normalize to lowercase, strip punctuation,
    remove stop words, drop words of length < 3, strip digits.
    """
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct = regex.sub(" ", text)  # delete stuff but leave at least a space to avoid clumping together
    words = nopunct.split(" ")
    words = [w for w in words if len(w) > 2]  # ignore a, an, to, at, be, ...
    words = [w.lower() for w in words]
    goodwords = [w for w in words if w not in ENGLISH_STOP_WORDS]
    return goodwords


def stemwords(words):
    """
    Given a list of tokens/words, return a new list with each word
    stemmed using a PorterStemmer.
    """
    stemmer = PorterStemmer()
    return [stemmer.stem(w) for w in words]


def tokenizer(text):
    return stemwords(tokenize(text))

In [ ]:
vectorizer = TfidfVectorizer(input='content',
                            analyzer='word',
#                             tokenizer=tokenizer,
                            stop_words='english',
                            decode_error='ignore')

dt_model = Pipeline([('vec', vectorizer),
                       ('clf', DecisionTreeClassifier(random_state=RANDOM_STATE))])

In [ ]:
dt_model.fit(train_data, train_target)

In [ ]:
predicted = dt_model.predict(test_data)
accuracy_score(predicted, test_target)

In [ ]:
grid_params = dict(vec__ngram_range=[(1,1), (1,2), (1,3)],
                   clf__criterion=['gini', 'entropy'])

In [ ]:
gs2 = GridSearchCV(estimator=dt_model,
                 param_grid=grid_params,
                 scoring='accuracy',
                 cv=5,
                 n_jobs=-1)

In [ ]:
gs2.fit(train_data, train_target)

In [ ]:
gs2.best_params_

In [ ]:
gs2.best_estimator_

In [ ]:
gs2.score(test_data, test_target)

## Optimized Model with Classification Report

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
vectorizer = TfidfVectorizer(input='content',
                            analyzer='word',
#                             tokenizer=tokenizer,
                            stop_words='english',
                            decode_error='ignore')

lsvc_model = Pipeline([('vec', vectorizer),
                       ('clf', LinearSVC(random_state=RANDOM_STATE, loss='hinge'))])

In [ ]:
lsvc_model.fit(train_data, train_target)

In [ ]:
predictions = lsvc_model.predict(test_data)
print(classification_report(test_target, predictions))